In [ ]:
!pip install opencv-contrib-python-headless
!pip install tensorflow==1.5
!pip install keras
!pip install https://github.com/OlafenwaMoses/ImageAI/releases/download/2.0.2/imageai-2.0.2-py3-none-any.whl
#!pip install sk-video
#!pip install pytube
#!pip install humanize

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import scipy
import pandas as pd

import cv2
#import skvideo.io
import imageio
#import pytube

import os
import time
import csv
import re
#import humanize
#from tqdm import tqdm

from skimage.measure import compare_mse, compare_ssim
from scipy.stats import wasserstein_distance
from scipy.spatial.distance import hamming
from imageai.Detection import ObjectDetection

Using TensorFlow backend.


In [4]:
save_path = '/home/idies/workspace/Storage/Cong/persistent/video/data'
data_path = '/home/idies/workspace/Storage/Cong/persistent/video/data'
result_path = '/home/idies/workspace/Storage/Cong/persistent/video/result'

In [4]:
def image_crop(h1, h2, l1, l2, original):
    return original[h1:h2, l1:l2, :]

In [4]:
def add_mask(img, coordinates):
    mask = np.zeros(img.shape, dtype = 'uint8')
    
    for coord in coordinates:
        cv2.rectangle(mask, coord[0], coord[1], (255, 255, 255), -1);
        
    maskedImg = cv2.bitwise_and(img, mask)
    return maskedImg

In [5]:
def detect_object(img, model='resnet50_coco_best_v2.0.1.h5', minimum_prob=80):
    detector = ObjectDetection()
    detector.setModelTypeAsRetinaNet()
    detector.setModelPath(os.path.join('/home/idies/workspace/Storage/Cong/persistent/video', model))
    detector.loadModel()
    detections = detector.detectObjectsFromImage(input_image=img, input_type='array', minimum_percentage_probability=minimum_prob)
    return detections

In [6]:
def get_FPS(video_loc):
    video = cv2.VideoCapture(video_loc)
    fps = video.get(cv2.CAP_PROP_FPS)
    return fps

In [7]:
def get_histogram(img):
    h, w = img.shape
    hist = [0.0] * 256
    for i in range(h):
        for j in range(w):
            hist[img[i, j]] += 1
    return np.array(hist) / (h * w)

In [8]:
def normalize_exposure(img):
    '''
    Normalize the exposure of an image.
    '''
    img = img.astype(int)
    hist = get_histogram(img)
    # get the sum of vals accumulated by each position in hist
    cdf = np.array([sum(hist[:i+1]) for i in range(len(hist))])
    # determine the normalization values for each unit of the cdf
    sk = np.uint8(255 * cdf)
    # normalize each position in the output image
    h, w = img.shape
    normalized = np.zeros_like(img)
    for i in range(0, h):
        for j in range(0, w):
            normalized[i, j] = sk[img[i, j]]
    return normalized.astype(int)

In [9]:
def compare_emd(img1, img2):
    img1 = normalize_exposure(img1)
    img2 = normalize_exposure(img2)
    return wasserstein_distance(get_histogram(img1), get_histogram(img2))

In [6]:
def compare_orb(img1, img2, threshold):
    orb = cv2.ORB_create()
    kp_1, desc_1 = orb.detectAndCompute(img1, None)
    kp_2, desc_2 = orb.detectAndCompute(img2, None)
    if desc_1 is None or desc_2 is None:
        return 0
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(desc_1, desc_2)
    if len(matches) == 0:
        return 0
    similar = [i for i in matches if i.distance < threshold]
    return len(similar) / len(matches)

In [11]:
def dhash(img, hashSize=8):
    # resize the input image, adding a single column (row) so we can compute the gradient
    resized_col = cv2.resize(img, (hashSize+1,hashSize))
    resized_row = cv2.resize(img, (hashSize,hashSize+1))
 
    # compute the (relative) gradient between adjacent pixels
    diff_col = resized_col[:,1:] > resized_col[:,:-1]
    diff_row = resized_row[1:,:] > resized_row[:-1,:]

    # convert the difference image to a hash
    return sum([2**i for (i, v) in enumerate(np.append(diff_col.flatten(), diff_row.flatten())) if v])

In [12]:
def compare_dhash(img1, img2, hashSize=8):
    h1 = [int(d) for d in str(int(dhash(img1)))]
    h2 = [int(d) for d in str(int(dhash(img2)))]
    
    if len(h1) == len(h2):
        dHash = hamming(h1, h2)
    else:
        dHash = 1
    return dHash

In [7]:
def compare_phash(img1, img2):
    phash = cv2.img_hash_PHash.create()
    return phash.compare(phash.compute(img1), phash.compute(img2))

In [8]:
allcarcoords = np.load(os.path.join(data_path, 'carcoords', 'allcarcoords.npy')).item()

In [56]:
def get_similar_image(image_df, method, ascending, idx, start, end, coords):
    results_dict = {}
            
    for i in range(start, min(end, image_df.shape[0])):
        
        img = imread(image_df['img_path'][idx])
        
        #i = i - image_df['img_index'][0]
        temp_img = imread(image_df['img_path'][i])
        
        if method == 'EMD' or method == 'ORB' or method == 'dHash':
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            temp_img = cv2.cvtColor(temp_img, cv2.COLOR_RGB2GRAY)
        
        #img = image_crop(h1, h2, l1, l2, img)
        img = add_mask(img, coords)
        #temp_img = image_crop(h1, h2, l1, l2, temp_img)
        temp_img = add_mask(temp_img, coords)
        
        detections = allcarcoords[idx][2]
        #detections = detect_object(img)
        carcoords = []
        for detection in detections:
            if detection.get('name') == 'car':
                temp = detection.get('box_points')
                carcoords.append(temp)
        
        tempdetections = allcarcoords[i][2]
        #tempdetections = detect_object(temp_img)
        for detection in tempdetections:
            if detection.get('name') == 'car':
                temp = detection.get('box_points')
                carcoords.append(temp)
        
        for carcoord in carcoords:
            cv2.rectangle(img,(carcoord[0],carcoord[1]),(carcoord[2],carcoord[3]),(0,0,0),-1);
            cv2.rectangle(temp_img,(carcoord[0],carcoord[1]),(carcoord[2],carcoord[3]),(0,0,0),-1);
        
        if method == 'MSE':
            results_dict[i] = [image_df['img_path'][i], image_df['img_index'][i], compare_mse(img, temp_img)]
        elif method == 'EMD':
            results_dict[i] = [image_df['img_path'][i], image_df['img_index'][i], compare_emd(img, temp_img)]
        elif method == 'ORB':
            results_dict[i] = [image_df['img_path'][i], image_df['img_index'][i], compare_orb(img, temp_img, threshold)]
        elif method == 'dHash':
            results_dict[i] = [image_df['img_path'][i], image_df['img_index'][i], compare_dhash(img, temp_img)]
        elif method == 'pHash':
            results_dict[i] = [image_df['img_path'][i], image_df['img_index'][i], compare_phash(img, temp_img)]
        else:
            results_dict[i] = [image_df['img_path'][i], image_df['img_index'][i], compare_ssim(img, temp_img, multichannel=True)]
    
    results_df = pd.DataFrame.from_dict(results_dict, orient='index')
    results_df.columns = ['img_path', 'img_index', method]
    results_df = results_df.sort_values(by=method, ascending=ascending)
    return results_df

In [9]:
def weighted_index(df, n, method, ascending):
    temp = df.copy()[:n]
    if ascending:
        temp['weight'] = (1/(temp[method]+1e-8)) / sum(1/(temp[method]+1e-8))
    else:
        temp['weight'] = temp[method] / sum(temp[method])
    return sum(temp['img_index']*temp['weight'])

In [10]:
threshold = 70

In [58]:
def lap_time2(initial_idx, end, fps, topn, df, method, ascending, coords):
    results_dict = {}
    start = initial_idx + np.round(fps*prior_time).astype(int)
    
    #img_idxs = np.arange(initial_idx, initial_idx+3).tolist()
    #img_idxs = np.arange(initial_idx+3, initial_idx+6).tolist()
    #img_idxs = np.arange(initial_idx+6, initial_idx+9).tolist()
    #img_idxs = np.arange(initial_idx+9, initial_idx+12).tolist()
    #img_idxs = np.arange(initial_idx+12, initial_idx+15).tolist()
    #img_idxs = np.arange(initial_idx+15, initial_idx+18).tolist()
    #img_idxs = np.arange(initial_idx+18, initial_idx+21).tolist()
    #img_idxs = np.arange(initial_idx+21, initial_idx+24).tolist()
    #img_idxs = np.arange(initial_idx+24, initial_idx+27).tolist()
    #img_idxs = np.arange(initial_idx+27, np.round(initial_idx+fps).astype(int)).tolist()
    
    #img_idxs = np.arange(initial_idx, initial_idx+5).tolist()
    #img_idxs = np.arange(initial_idx+5, initial_idx+10).tolist()
    #img_idxs = np.arange(initial_idx+10, initial_idx+15).tolist()
    #img_idxs = np.arange(initial_idx+15, initial_idx+20).tolist()
    #img_idxs = np.arange(initial_idx+20, initial_idx+25).tolist()
    img_idxs = np.arange(initial_idx+25, np.round(initial_idx+fps).astype(int)).tolist()
    
    #img_idxs = np.arange(initial_idx, initial_idx+10).tolist()
    #img_idxs = np.arange(initial_idx+10, initial_idx+20).tolist()
    #img_idxs = np.arange(initial_idx+20, np.round(initial_idx+fps).astype(int)).tolist()
    
    #img_idxs = np.choice(np.arange(initial_idx, np.round(initial_idx+fps).astype(int)), 10).tolist()
    
    for idx in img_idxs:
        #img = imread(df['img_path'][idx])
        start_time = time.time()
        similar_imgs = get_similar_image(df, method, ascending, idx, start, end, coords)
        elapsedTime = time.time() - start_time
        results_dict[idx] = [df['img_path'][idx], df['img_index'][idx], 
                             (np.mean(similar_imgs.img_index[similar_imgs[method]==(similar_imgs[method].iloc[0])])-df['img_index'][idx])/fps, 
                             (weighted_index(similar_imgs, topn, method, ascending)-df['img_index'][idx])/fps, 
                             elapsedTime, method]
    
    results_df = pd.DataFrame.from_dict(results_dict, orient='index')
    results_df.columns = ['img_path', 'img_index', 'lap_time', 'weighted_lap_time', 'elapsedTime', 'method']
    return results_df

In [6]:
df = pd.read_csv(os.path.join(data_path, 'train091108_raw.csv'))

initial_idx = 21681
end = 26146
prior_time = 60

fps = 29.93930396046777
topn = 5

method = 'pHash'
ascending = True

coords = [[(0, 600), (1920, 740)], [(300, 0), (1920, 740)]]

In [ ]:
test = lap_time2(initial_idx, end, fps, topn, df, method, ascending, coords)
test['lap'] = 9
test.to_csv(os.path.join(result_path, '091108', 'lap9_pHash_6.csv'), index=False)

In [52]:
temp1 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_1.csv'))
temp2 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_2.csv'))
temp3 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_3.csv'))
temp4 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_4.csv'))
temp5 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_5.csv'))
temp6 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_6.csv'))
# temp7 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_7.csv'))
# temp8 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_8.csv'))
# temp9 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_9.csv'))
# temp10 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_10.csv'))
temp = temp1.append(temp2)
temp = temp.append(temp3)
temp = temp.append(temp4)
temp = temp.append(temp5)
temp = temp.append(temp6)
# temp = temp.append(temp7)
# temp = temp.append(temp8)
# temp = temp.append(temp9)
# temp = temp.append(temp10)

# initial_idx
print(int(temp['lap_time'].iloc[0] * fps) + initial_idx)

# start
print(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int))

# end
print(int(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int) + np.mean(temp['lap_time']) * fps))

24359
26155
28828


In [53]:
temp.to_csv(os.path.join(result_path, '091108', 'lap9_pHash.csv'), index=False)

In [54]:
temp = pd.read_csv('./result/091108/lap9_pHash.csv')
np.median(temp['lap_time'])

89.54227315615418

In [51]:
# initial_idx = 0
# end = 3600

# initial_idx = 2761
# end = 7315

# initial_idx = 5519
# end = 10077

# initial_idx = 8237
# end = 12752

# initial_idx = 10933
# end = 15432

# initial_idx = 13626
# end = 18055

# initial_idx = 16316
# end = 20747

# initial_idx = 18984
# end = 23416

# initial_idx = 21681
# end = 26146

In [47]:
# initial_idx = 0
# end = 3600

# initial_idx = 2763
# end = 7325

# initial_idx = 5519
# end = 10077

# initial_idx = 8244
# end = 12789

# initial_idx = 10932
# end = 15434

# initial_idx = 13619
# end = 18073

# initial_idx = 16313
# end = 20790

# initial_idx = 18982
# end = 23434

# initial_idx = 21677
# end = 26178

In [239]:
# df = pd.read_csv(os.path.join(data_path, 'train153240_raw.csv'))

# initial_idx = 27877
# end = 31802
# prior_time = 60

# fps = 29.93943500598125
# topn = 5

# method = 'ORB'
# ascending = False

# coords = [[(0, 640), (1920, 810)], [(320, 0), (1920, 810)]]

In [240]:
# test = lap_time2(initial_idx, end, fps, topn, df, method, ascending, coords)
# test['lap'] = 11
# test.to_csv(os.path.join(result_path, '153240', 'lap11_ORB.csv'), index=False)

In [272]:
# temp1 = pd.read_csv(os.path.join(result_path, '153240', 'lap10_ORB_1.csv'))
# temp2 = pd.read_csv(os.path.join(result_path, '153240', 'lap10_ORB_2.csv'))
# temp3 = pd.read_csv(os.path.join(result_path, '153240', 'lap10_ORB_3.csv'))
# temp4 = pd.read_csv(os.path.join(result_path, '153240', 'lap10_ORB_4.csv'))
# temp5 = pd.read_csv(os.path.join(result_path, '153240', 'lap10_ORB_5.csv'))
# temp6 = pd.read_csv(os.path.join(result_path, '153240', 'lap10_ORB_6.csv'))
# temp7 = pd.read_csv(os.path.join(result_path, '153240', 'lap10_ORB_7.csv'))
# temp8 = pd.read_csv(os.path.join(result_path, '153240', 'lap10_ORB_8.csv'))
# temp9 = pd.read_csv(os.path.join(result_path, '153240', 'lap10_ORB_9.csv'))
# temp10 = pd.read_csv(os.path.join(result_path, '153240', 'lap10_ORB_10.csv'))
# temp = temp1.append(temp2)
# temp = temp.append(temp3)
# temp = temp.append(temp4)
# temp = temp.append(temp5)
# temp = temp.append(temp6)
# temp = temp.append(temp7)
# temp = temp.append(temp8)
# temp = temp.append(temp9)
# temp = temp.append(temp10)

# # initial_idx
# print(int(temp['lap_time'].iloc[0] * fps) + initial_idx)

# # start
# print(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int))

# # end
# print(int(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int) + np.mean(temp['lap_time']) * fps))

27877
29673
31802


In [273]:
# temp.to_csv(os.path.join(result_path, '153240', 'lap10_ORB.csv'), index=False)

In [274]:
# temp = pd.read_csv('./result/153240/lap10_ORB.csv')
# np.mean(temp['lap_time'])

71.12692672973195

In [271]:
# initial_idx = 0
# end = 3600

# initial_idx = 2784
# end = 7323

# initial_idx = 5524
# end = 10893

# initial_idx = 8236
# end = 13824

# initial_idx = 10935
# end = 15795

# initial_idx = 15395
# end = 20829

# initial_idx = 18063
# end = 22975

# initial_idx = 20749
# end = 25217

# initial_idx = 23390
# end = 27826

# initial_idx = 26043
# end = 30431

In [106]:
# df = pd.read_csv(os.path.join(data_path, 'train153240_raw.csv'))

# initial_idx = 26985
# end = 31456
# prior_time = 60

# fps = 29.93943500598125
# topn = 5

# method = 'MSE'
# ascending = True

# coords = [[(0, 640), (1920, 810)], [(320, 0), (1920, 810)]]

In [191]:
# test = lap_time2(initial_idx, end, fps, topn, df, method, ascending, coords)
# test['lap'] = 11
# test.to_csv(os.path.join(result_path, '153240', 'lap11_MSE_6.csv'), index=False)

In [201]:
# temp1 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_MSE_1.csv'))
# temp2 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_MSE_2.csv'))
# temp3 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_MSE_3.csv'))
# temp4 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_MSE_4.csv'))
# temp5 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_MSE_5.csv'))
# temp6 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_MSE_6.csv'))
# temp = temp1.append(temp2)
# temp = temp.append(temp3)
# temp = temp.append(temp4)
# temp = temp.append(temp5)
# temp = temp.append(temp6)

# # initial_idx
# print(int(temp['lap_time'].iloc[0] * fps) + initial_idx)

# # start
# print(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int))

# # end
# print(int(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int) + np.mean(temp['lap_time']) * fps))

5410
7206
9835


In [202]:
# temp.to_csv(os.path.join(result_path, '153240', 'lap11_MSE.csv'), index=False)

In [203]:
# temp = pd.read_csv('./result/153240/lap11_MSE.csv')
# np.mean(temp['lap_time'])

87.81394837527037

In [190]:
# initial_idx = 0
# end = 3600

# initial_idx = 2786
# end = 7363

# initial_idx = 5527
# end = 9348

# initial_idx = 8236
# end = 12741

# initial_idx = 10940
# end = 15438

# initial_idx = 13666
# end = 18190

# initial_idx = 16355
# end = 20839

# initial_idx = 19039
# end = 23520

# initial_idx = 21693
# end = 26140

# initial_idx = 24297
# end = 28744

In [106]:
# df = pd.read_csv(os.path.join(data_path, 'train153240_raw.csv'))

# initial_idx = 29332
# end = 31230
# prior_time = 5

# fps = 29.93943500598125
# topn = 5

# method = 'pHash'
# ascending = True

# coords = [[(0, 640), (1920, 810)], [(320, 0), (1920, 810)]]

In [191]:
# test = lap_time2(initial_idx, end, fps, topn, df, method, ascending, coords)
# test['lap'] = 11
# test.to_csv(os.path.join(result_path, '153240', 'lap11_pHash_6.csv'), index=False)

In [117]:
# temp1 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_pHash_1.csv'))
# temp2 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_pHash_2.csv'))
# temp3 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_pHash_3.csv'))
# temp4 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_pHash_4.csv'))
# temp5 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_pHash_5.csv'))
# temp6 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_pHash_6.csv'))
# temp = temp1.append(temp2)
# temp = temp.append(temp3)
# temp = temp.append(temp4)
# temp = temp.append(temp5)
# temp = temp.append(temp6)

# # initial_idx
# print(int(temp['lap_time'].iloc[0] * fps) + initial_idx)

# # start
# print(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int))

# # end
# print(int(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int) + np.mean(temp['lap_time']) * fps))

26862
28658
28856


In [118]:
# temp.to_csv(os.path.join(result_path, '153240', 'lap11_pHash.csv'), index=False)

In [119]:
# temp = pd.read_csv('./result/153240/lap11_pHash.csv')
# np.mean(temp['lap_time'])

6.62242512682876

In [112]:
# initial_idx = 0
# end = 3600

# initial_idx = 2780
# end = 7348

# initial_idx = 5521
# end = 10760

# initial_idx = 8230
# end = 12989

# initial_idx = 12658
# end = 17665

# initial_idx = 15865
# end = 20419

# initial_idx = 18479
# end = 22918

# initial_idx = 21305
# end = 25793

# initial_idx = 24040
# end = 28484

# initial_idx = 26709
# end = 31230

In [57]:
# df = pd.read_csv(os.path.join(data_path, 'train091108_raw.csv'))

# initial_idx = 21526
# end = 26018
# prior_time = 60

# fps = 29.93930396046777
# topn = 5

# method = 'pHash'
# ascending = True

# coords = [[(0, 600), (1920, 740)], [(300, 0), (1920, 740)]]

In [58]:
# test = lap_time2(initial_idx, end, fps, topn, df, method, ascending, coords)
# test['lap'] = 9
# test.to_csv(os.path.join(result_path, '091108', 'lap9_pHash_6.csv'), index=False)

In [68]:
# temp1 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_1.csv'))
# temp2 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_2.csv'))
# temp3 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_3.csv'))
# temp4 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_4.csv'))
# temp5 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_5.csv'))
# temp6 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_6.csv'))
# temp = temp1.append(temp2)
# temp = temp.append(temp3)
# temp = temp.append(temp4)
# temp = temp.append(temp5)
# temp = temp.append(temp6)

# # initial_idx
# print(int(temp['lap_time'].iloc[0] * fps) + initial_idx)

# # start
# print(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int))

# # end
# print(int(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int) + np.mean(temp['lap_time']) * fps))

21514
23310
25991


In [69]:
# temp.to_csv(os.path.join(result_path, '091108', 'lap9_pHash.csv'), index=False)

In [70]:
# temp = pd.read_csv('./result/091108/lap9_pHash.csv')
# np.mean(temp['lap_time'])

89.56249926282501

In [64]:
# initial_idx = 0
# end = 3600

# initial_idx = 2767
# end = 7323

# initial_idx = 5526
# end = 10083

# initial_idx = 8247
# end = 12763

# initial_idx = 10944
# end = 15439

# initial_idx = 13632
# end = 18064

# initial_idx = 16320
# end = 20752

# initial_idx = 18831
# end = 23219

In [188]:
# df = pd.read_csv(os.path.join(data_path, 'train153240_raw.csv'))

# initial_idx = 27001
# end = 33056

# fps = 29.93943500598125
# topn = 5

# method = 'ORB'
# ascending = False

# h1 = 400
# h2 = 810
# l1 = 250
# l2 = 1920

In [191]:
# test = lap_time2(initial_idx, end, fps, topn, df, method, ascending, h1, h2, l1, l2)
# test['lap'] = 11
# test.to_csv(os.path.join(result_path, '153240', 'lap11_ORB_6.csv'), index=False)

In [192]:
# temp1 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_ORB_1.csv'))
# temp2 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_ORB_2.csv'))
# temp3 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_ORB_3.csv'))
# temp4 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_ORB_4.csv'))
# temp5 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_ORB_5.csv'))
# temp6 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_ORB_6.csv'))
# temp = temp1.append(temp2)
# temp = temp.append(temp3)
# temp = temp.append(temp4)
# temp = temp.append(temp5)
# temp = temp.append(temp6)

# # initial_idx
# print(int(temp['lap_time'].iloc[0] * fps) + initial_idx)

# # start
# print(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int))

# # end
# print(int(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int) + np.mean(temp['lap_time']) * fps))

29631
30529
33110


In [193]:
# temp.to_csv(os.path.join(result_path, '153240', 'lap11_ORB.csv'), index=False)

In [194]:
# temp = pd.read_csv('./result/153240/lap11_ORB.csv')
# np.mean(temp['lap_time'])

86.22295865472896

In [155]:
# initial_idx = 0
# end = 3600

# initial_idx = 2784
# end = 7362

# initial_idx = 5524
# end = 10062

# initial_idx = 8236
# end = 12744

# initial_idx = 10939
# end = 15436

# initial_idx = 13666
# end = 18190

# initial_idx = 18103
# end = 22645

# initial_idx = 20785
# end = 25260

# initial_idx = 23429
# end = 27863

# initial_idx = 26084
# end = 30523

In [19]:
# df = pd.read_csv(os.path.join(data_path, 'train153240_raw.csv'))

# initial_idx = 27001
# end = 31470

# fps = 29.93943500598125
# topn = 5

# method = 'pHash'
# ascending = True

# h1 = 400
# h2 = 810
# l1 = 250
# l2 = 1920

In [ ]:
# test = lap_time2(initial_idx, end, fps, topn, df, method, ascending, h1, h2, l1, l2)
# test['lap'] = 11
# test.to_csv(os.path.join(result_path, '153240', 'lap11_pHash_6.csv'), index=False)

In [63]:
# temp1 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_pHash_1.csv'))
# temp2 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_pHash_2.csv'))
# temp3 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_pHash_3.csv'))
# temp4 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_pHash_4.csv'))
# temp5 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_pHash_5.csv'))
# temp6 = pd.read_csv(os.path.join(result_path, '153240', 'lap11_pHash_6.csv'))
# temp = temp1.append(temp2)
# temp = temp.append(temp3)
# temp = temp.append(temp4)
# temp = temp.append(temp5)
# temp = temp.append(temp6)

# # initial_idx
# print(int(temp['lap_time'].iloc[0] * fps) + initial_idx)

# # start
# print(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int))

# # end
# print(int(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int) + np.mean(temp['lap_time']) * fps))

29623
31419
34028


In [64]:
# temp.to_csv(os.path.join(result_path, '153240', 'lap11_pHash.csv'), index=False)

In [65]:
# temp = pd.read_csv('./result/153240/lap11_pHash.csv')
# np.mean(temp['lap_time'])

87.16374683797424

In [62]:
# initial_idx = 0
# end = 3600

# initial_idx = 2791
# end = 7361

# initial_idx = 5515
# end = 10237

# initial_idx = 8214
# end = 12717

# initial_idx = 10923
# end = 15425

# initial_idx = 13650
# end = 18175

# initial_idx = 16338
# end = 20818

# initial_idx = 19028
# end = 23511

# initial_idx = 21678
# end = 26129

# initial_idx = 24334
# end = 28778

# initial_idx = 27001
# end = 31470

In [61]:
# df = pd.read_csv(os.path.join(data_path, 'train091108_raw.csv'))

# initial_idx = 21661
# end = 26156

# fps = 29.93930396046777
# topn = 5

# method = 'ORB'
# ascending = False

# h1 = 400
# h2 = 760
# l1 = 250
# l2 = 1920

In [62]:
# test = lap_time2(initial_idx, end, fps, topn, df, method, ascending, h1, h2, l1, l2)
# test['lap'] = 9
# test.to_csv(os.path.join(result_path, '091108', 'lap9_ORB_6.csv'), index=False)

In [79]:
# temp1 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_ORB_1.csv'))
# temp2 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_ORB_2.csv'))
# temp3 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_ORB_3.csv'))
# temp4 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_ORB_4.csv'))
# temp5 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_ORB_5.csv'))
# temp6 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_ORB_6.csv'))
# temp = temp1.append(temp2)
# temp = temp.append(temp3)
# temp = temp.append(temp4)
# temp = temp.append(temp5)
# temp = temp.append(temp6)

# # initial_idx
# print(int(temp['lap_time'].iloc[0] * fps) + initial_idx)

# # start
# print(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int))

# # end
# print(int(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int) + np.mean(temp['lap_time']) * fps))

21640
23436
26115


In [80]:
# temp.to_csv(os.path.join(result_path, '091108', 'lap9_ORB.csv'), index=False)

In [81]:
# temp = pd.read_csv('./result/091108/lap9_ORB.csv')
# np.mean(temp['lap_time'])

89.48437824531652

In [75]:
# initial_idx = 0
# end = 3600

# initial_idx = 2756
# end = 7308

# initial_idx = 5511
# end = 10067

# initial_idx = 8227
# end = 12740

# initial_idx = 10917
# end = 15406

# initial_idx = 13607
# end = 18091

# initial_idx = 16297
# end = 20783

# initial_idx = 18963
# end = 23425

In [20]:
# df = pd.read_csv(os.path.join(data_path, 'train091108_raw.csv'))

# initial_idx = 21684
# end = 26149

# fps = 29.93930396046777
# topn = 5

# method = 'pHash'
# ascending = True

# h1 = 400
# h2 = 760
# l1 = 250
# l2 = 1920

In [ ]:
# test = lap_time2(initial_idx, end, fps, topn, df, method, ascending, h1, h2, l1, l2)
# test['lap'] = 9
# test.to_csv(os.path.join(result_path, '091108', 'lap9_pHash_3.csv'), index=False)

In [81]:
# temp1 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_1.csv'))
# temp2 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_2.csv'))
# temp3 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_3.csv'))
# temp = temp1.append(temp2)
# temp = temp.append(temp3)

# # initial_idx
# print(int(temp['lap_time'].iloc[0] * fps) + initial_idx)

# # start
# print(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int))

# # end
# print(int(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int) + np.mean(temp['lap_time']) * fps))

24359
26155
28842


In [82]:
# temp.to_csv(os.path.join(result_path, '091108', 'lap9_pHash.csv'), index=False)

In [84]:
# temp = pd.read_csv('./result/091108/lap9_pHash.csv')
# np.mean(temp['lap_time'])

89.77719289051473

In [80]:
# initial_idx = 0
# end = 3600

# initial_idx = 2766
# end = 7323

# initial_idx = 5524
# end = 10080

# initial_idx = 8245
# end = 12760

# initial_idx = 10937
# end = 15398

# initial_idx = 13623
# end = 18104

# initial_idx = 16316
# end = 20805

# initial_idx = 18983
# end = 23449

# initial_idx = 21684
# end = 26149

In [20]:
# df = pd.read_csv(os.path.join(data_path, 'train091108_raw.csv'))

# initial_idx = 21229
# end = 26502

# fps = 29.93930396046777
# topn = 5

# method = 'dHash'
# ascending = True

# h1 = 400
# h2 = 760
# l1 = 250
# l2 = 1920

In [ ]:
# test = lap_time2(initial_idx, end, fps, topn, df, method, ascending, h1, h2, l1, l2)
# test['lap'] = 9
# test.to_csv(os.path.join(result_path, '091108', 'lap9_dHash_1.csv'), index=False)

In [116]:
# temp1 = pd.read_csv(os.path.join(result_path, '091108', 'lap8_dHash_1.csv'))
# temp2 = pd.read_csv(os.path.join(result_path, '091108', 'lap8_dHash_2.csv'))
# temp3 = pd.read_csv(os.path.join(result_path, '091108', 'lap8_dHash_3.csv'))
# temp4 = pd.read_csv(os.path.join(result_path, '091108', 'lap8_dHash_4.csv'))
# temp5 = pd.read_csv(os.path.join(result_path, '091108', 'lap8_dHash_5.csv'))
# temp6 = pd.read_csv(os.path.join(result_path, '091108', 'lap8_dHash_6.csv'))
# temp = temp1.append(temp2)
# temp = temp.append(temp3)
# temp = temp.append(temp4)
# temp = temp.append(temp5)
# temp = temp.append(temp6)

# # initial_idx
# print(int(temp['lap_time'].iloc[0] * fps) + initial_idx)

# # start
# print(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int))

# # end
# print(int(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int) + np.mean(temp['lap_time']) * fps))

24062
25858
28276


In [118]:
# temp.to_csv(os.path.join(result_path, '091108', 'lap8_dHash.csv'), index=False)

In [119]:
# temp = pd.read_csv('./result/091108/lap8_dHash.csv')
# np.mean(temp['lap_time'])

80.79568816498539

In [115]:
# initial_idx = 0
# end = 3600

# initial_idx = 2710
# end = 7118

# initial_idx = 4946
# end = 9848

# initial_idx = 8902
# end = 13998

# initial_idx = 11070
# end = 16267

# initial_idx = 13773
# end = 18853

# initial_idx = 18401
# end = 23864

# initial_idx = 21229
# end = 26502